# Build a cuisine recommender

In [1]:
# %pip install skl2onnx
import pandas as pd 


In [3]:
data = pd.read_csv('./data/cleaned_cuisines.csv')
data.head()

Unnamed: 0 cuisine  almond  angelica  anise  anise_seed  apple  \
0           0  indian       0         0      0           0      0   
1           1  indian       1         0      0           0      0   
2           2  indian       0         0      0           0      0   
3           3  indian       0         0      0           0      0   
4           4  indian       0         0      0           0      0   

   apple_brandy  apricot  armagnac  ...  whiskey  white_bread  white_wine  \
0             0        0         0  ...        0            0           0   
1             0        0         0  ...        0            0           0   
2             0        0         0  ...        0            0           0   
3             0        0         0  ...        0            0           0   
4             0        0         0  ...        0            0           0   

   whole_grain_wheat_flour  wine  wood  yam  yeast  yogurt  zucchini  
0                        0     0     0    0      0       0         0  
1                        0     0     0    0      0       0         0  
2                        0     0     0    0      0       0         0  
3                        0     0     0    0      0       0         0  
4                        0     0     0    0      0       1         0  

[5 rows x 382 columns]

In [4]:
X = data.iloc[:,2:]
X.head()

almond  angelica  anise  anise_seed  apple  apple_brandy  apricot  \
0       0         0      0           0      0             0        0   
1       1         0      0           0      0             0        0   
2       0         0      0           0      0             0        0   
3       0         0      0           0      0             0        0   
4       0         0      0           0      0             0        0   

   armagnac  artemisia  artichoke  ...  whiskey  white_bread  white_wine  \
0         0          0          0  ...        0            0           0   
1         0          0          0  ...        0            0           0   
2         0          0          0  ...        0            0           0   
3         0          0          0  ...        0            0           0   
4         0          0          0  ...        0            0           0   

   whole_grain_wheat_flour  wine  wood  yam  yeast  yogurt  zucchini  
0                        0     0     0    0      0       0         0  
1                        0     0     0    0      0       0         0  
2                        0     0     0    0      0       0         0  
3                        0     0     0    0      0       0         0  
4                        0     0     0    0      0       1         0  

[5 rows x 380 columns]

In [5]:
y = data[['cuisine']]
y.head()

cuisine
0  indian
1  indian
2  indian
3  indian
4  indian

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [8]:
model = SVC(kernel='linear', C=10, probability=True,random_state=0)
model.fit(X_train,y_train.values.ravel())

SVC(C=10, kernel='linear', probability=True, random_state=0)

In [9]:
y_pred = model.predict(X_test)

In [10]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     chinese       0.68      0.73      0.70       234
      indian       0.90      0.82      0.86       240
    japanese       0.74      0.78      0.76       243
      korean       0.84      0.71      0.77       230
        thai       0.76      0.85      0.80       252

    accuracy                           0.78      1199
   macro avg       0.78      0.78      0.78      1199
weighted avg       0.78      0.78      0.78      1199



## Convert model to Onnx

In [11]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 380]))]
options = {id(model): {'nocl': True, 'zipmap': False}}

In [12]:
onx = convert_sklearn(model, initial_types=initial_type, options=options)
with open("./model.onnx", "wb") as f:
    f.write(onx.SerializeToString())